In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


In [0]:
!ls 


In [0]:
!mkdir dataset

In [0]:
!cp "gdrive/My Drive/datasets/output_immagini11_frames.zip" dataset/

In [0]:
!unzip dataset/output_immagini11_frames.zip

In [0]:
# import necessary libraries
import torch
import torchvision
from torchvision import transforms as T
import torch.nn.functional as F

# Library needed for visualization purposes
from tensorboardcolab import TensorBoardColab

# Instantiate visualizer
tb = TensorBoardColab(graph_path='./log')



In [0]:
import torch.nn as nn

class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.MaxPool2d(2, 2, 1),
            nn.Conv2d(64, 128, 3, 1, 1),
         
            nn.MaxPool2d(2, 2, 1),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.Conv2d(256, 256, 3, 1, 1),

            nn.MaxPool2d(2, 2, 1),
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.MaxPool2d(2, 2, 1),

            nn.Conv2d(512, 512, 3, 1, 1),
            nn.MaxPool2d(2, 2, 1),
            nn.MaxPool2d(2, 2, 1),

        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x



In [0]:
import torch.nn as nn

class VGG2(nn.Module):
    def __init__(self):
        super(VGG2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 512, 3, 1, 1),
            nn.MaxPool2d(2, 2, 1),
            nn.MaxPool2d(2, 2, 1),
            nn.ReLU(),
            
            nn.Conv2d(512, 256, 3, 1, 1),
            nn.MaxPool2d(2, 2, 1),
            nn.Conv2d(256, 128, 3, 1, 1),
            nn.MaxPool2d(2, 2, 1),
            nn.Dropout(0.3),
            
            nn.Conv2d(128, 64, 3, 1, 1),
            nn.Conv2d(64, 8, 3, 1, 1),
            nn.MaxPool2d(2, 2, 1),
            nn.ReLU()

        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x




In [0]:
class Dense_Block(nn.Module):
    def __init__(self, in_channels):
        super(Dense_Block, self).__init__()
        self.relu = nn.ReLU(inplace = True)
        self.bn = nn.BatchNorm2d(num_features = in_channels)

        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.conv4 = nn.Conv2d(in_channels = 96, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.conv5 = nn.Conv2d(in_channels = 128, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
    def forward(self, x):
        bn = self.bn(x) 
        conv1 = self.relu(self.conv1(bn))
        conv2 = self.relu(self.conv2(conv1))
        # Concatenate in channel dimension
        c2_dense = self.relu(torch.cat([conv1, conv2], 1))
        conv3 = self.relu(self.conv3(c2_dense))
        c3_dense = self.relu(torch.cat([conv1, conv2, conv3], 1))
        conv4 = self.relu(self.conv4(c3_dense)) 
        c4_dense = self.relu(torch.cat([conv1, conv2, conv3, conv4], 1))
        conv5 = self.relu(self.conv5(c4_dense))
        c5_dense = self.relu(torch.cat([conv1, conv2, conv3, conv4, conv5], 1))
        return c5_dense

class Transition_Layer(nn.Module): 
    def __init__(self, in_channels, out_channels):
        super(Transition_Layer, self).__init__() 

        self.relu = nn.ReLU(inplace = True) 
        self.bn = nn.BatchNorm2d(num_features = out_channels) 
        self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 1, bias = False) 
        self.avg_pool = nn.AvgPool2d(kernel_size = 2, stride = 2, padding = 0) 
    def forward(self, x): 
        bn = self.bn(self.relu(self.conv(x))) 
        out = self.avg_pool(bn) 
        return out 
    
class DenseNet(nn.Module): 
    def __init__(self, nr_classes): 
        super(DenseNet, self).__init__() 

        self.lowconv = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 7, padding = 3, bias = False) 
        self.relu = nn.ReLU()

        # Make Dense Blocks 
        self.denseblock1 = self._make_dense_block(Dense_Block, 64) 
        self.denseblock2 = self._make_dense_block(Dense_Block, 128)
        self.denseblock3 = self._make_dense_block(Dense_Block, 64)
        # Make transition Layers 
        self.transitionLayer1 = self._make_transition_layer(Transition_Layer, in_channels = 160, out_channels = 128) 
        self.transitionLayer2 = self._make_transition_layer(Transition_Layer, in_channels = 160, out_channels = 64) 
        self.transitionLayer3 = self._make_transition_layer(Transition_Layer, in_channels = 160, out_channels = 64)
        # Classifier 
        self.bn = nn.BatchNorm2d(num_features = 64) 
        self.pre_classifier = nn.Linear(64*8*8, 128) 
        self.classifier = nn.Linear(128, nr_classes)
 
    def _make_dense_block(self, block, in_channels): 
        layers = [] 
        layers.append(block(in_channels)) 
        return nn.Sequential(*layers) 
    def _make_transition_layer(self, layer, in_channels, out_channels): 
        modules = [] 
        modules.append(layer(in_channels, out_channels)) 
        return nn.Sequential(*modules) 
    def forward(self, x): 
        out = self.relu(self.lowconv(x)) 
        #print('OUT 1', out.shape)

        out = self.denseblock1(out) 
        #print('OUT 2', out.shape)

        out = self.transitionLayer1(out)
        #print('OUT 3', out.shape)

        out = self.denseblock2(out)
        #print('OUT 4', out.shape)

        out = self.transitionLayer2(out) 
        #print('OUT 5', out.shape)


        out = self.denseblock3(out) 
        #print('OUT 6', out.shape)

        out = self.transitionLayer3(out)
        #print('OUT 7', out.shape)


        out = self.bn(out)
        #print('out 8', out.shape)
        out = out.view(-1, 64*8*8) 
        #print('out 8', out.shape)


        out = self.pre_classifier(out) 
        out = self.classifier(out)
        return out

In [0]:
class BatchNorm(nn.Module):
    def __init__(self):
        super(BatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=20,
                               kernel_size=5,
                               stride=1)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.dense1 = nn.Linear(in_features=50*61*61, out_features=64)
        self.dense1_bn = nn.BatchNorm1d(64)
        #self.dense2 = nn.Linear(64, 32)
        self.dense3 = nn.Linear(64, 8)


    def forward(self, x):
        #print(x.shape)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        #print('1:',x.shape)

        x = F.relu(F.max_pool2d(self.conv2_bn(self.conv2(x)), 2))
        #print('2:',x.shape)

        x = x.view(-1, 50*61*61) #reshape
        #print('3:',x.shape)

        x = F.relu(self.dense1_bn(self.dense1(x)))
        #print('4:',x.shape)

        #x = F.relu(self.dense2(x))
        x = F.relu(self.dense3(x))
        return F.log_softmax(x)

### Define cost function

In [0]:
def get_cost_function():
    use_gpu = torch.cuda.is_available()
    #cost_function = torch.nn.CrossEntropyLoss()
    cost_function = nn.CrossEntropyLoss().cuda() if use_gpu else nn.CrossEntropyLoss()
    return cost_function

### Define the optimizer

In [0]:
def get_optimizer(net, lr, momentum):
    #optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
    #optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=wd, amsgrad=False)
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, nesterov = False)
    return optimizer
  


### Train and test functions

In [0]:
def test(net, data_loader, cost_function, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.

    cumulative_accuracy = 0.

    net.eval() # Strictly needed if network contains layers which has different behaviours between train and tes
    with torch.no_grad():
        for batch_idx, (inputs, targets,path) in enumerate(data_loader):
            # Load data into GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = net(inputs)

            # Apply the loss
            loss = cost_function(outputs, targets)

            # Better print something
            samples+=inputs.shape[0]
            cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
            _, predicted = outputs.max(1)
            cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss/samples, cumulative_accuracy/samples*100


def train(net,data_loader,optimizer,cost_function, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.
    cumulative_accuracy = 0.
    net.train() # Strictly needed if network contains layers which has different behaviours between train and test
    for batch_idx, (inputs, targets,path) in enumerate(data_loader):
        # Load data into GPU
        inputs = inputs.to(device)
        targets = targets.to(device)
      
        # Forward pass
        outputs = net(inputs)

        # Apply the loss
        loss = cost_function(outputs,targets)

        # Reset the optimizer

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()

        optimizer.zero_grad()

        # Better print something, no?
        samples+=inputs.shape[0]
        cumulative_loss += loss.item()
        _, predicted = outputs.max(1)
        cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss/samples, cumulative_accuracy/samples*100

In [0]:
import torch
from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path



In [0]:
def get_data(batch_size, test_batch_size=64):
  
  # Prepare data transformations and then combine them sequentially
    transform = list()
    #transform.append(T.Grayscale(num_output_channels=1))
    #transform.append(T.Scale((128)))
    transform.append(T.ToTensor())  
    #transform.append(T.Normalize([0.485, 0.456, 0.406],
                                # [0.229, 0.224, 0.225]))      # Normalizes the Tensors between [-1, 1]
    transform = T.Compose(transform)                          # Composes the above transformations into one.

    # Load data
    print('Start-Loading-Transform')
    #imagenet_data = torchvision.datasets.ImageFolder('./output_immagini11_frames/', transform=transform)
    imagenet_data = ImageFolderWithPaths('./output_immagini11_frames/', transform=transform)
    print('Finish transform')
    
    # Create train and test splits
    # We will create a 80:20 % train:test split
    num_samples = len(imagenet_data)
    training_samples = int(num_samples*0.7+1)
    test_samples = num_samples - training_samples

    training_data, test_data = torch.utils.data.random_split(imagenet_data, [training_samples, test_samples])
    
    num_samples = training_samples
    training_samples = int(num_samples*0.5+1)
    validation_samples = num_samples - training_samples
    
    training_data, validation_data = torch.utils.data.random_split(training_data, [training_samples, validation_samples])    
 
    #name_train = list()
    #for name in training_data:
    #  name_train.append(name[2])
    #name_val = list()
    #for name in validation_data:
    #  name_val.append(name[2])
    #name_test = list()
    #for name in test_data:
    #  name_test.append(name[2])
    
    # Initialize dataloaders
    train_loader = torch.utils.data.DataLoader(training_data, batch_size = batch_size, shuffle = True)
    val_loader = torch.utils.data.DataLoader(validation_data, batch_size = batch_size, shuffle = False)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = False)

    return train_loader, val_loader, test_loader#, name_train, name_val, name_test

In [0]:
import time
import torch.optim as optim
from torch.autograd import Variable
import torchvision.models as models


device='cuda:0'
#params
batch_size = 32
momentum = 0.9
learning_rate = 0.001
nr_classes = 8
num_epochs = 10

train_loader, val_loader, test_loader = get_data(batch_size)

#net = VGG().to(device)

# GPU flag
use_gpu = torch.cuda.is_available()
densenet = DenseNet(nr_classes)
#densenet = BatchNorm()
if use_gpu:
  densenet.cuda()

optimizer = get_optimizer(densenet, learning_rate, momentum)
cost_function = get_cost_function()



print('Before training:')
train_loss, train_accuracy = test(densenet, train_loader, cost_function)
val_loss, val_accuracy = test(densenet, val_loader, cost_function)
test_loss, test_accuracy = test(densenet, test_loader, cost_function)

print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
print('-----------------------------------------------------')

for e in range(num_epochs):
  train_loss, train_accuracy = train(densenet, train_loader, optimizer, cost_function)
  val_loss, val_accuracy = test(densenet, val_loader, cost_function)
  print('Epoch: {:d}'.format(e+1))
  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('-----------------------------------------------------')

print('After training:')
train_loss, train_accuracy = test(densenet, train_loader, cost_function)
val_loss, val_accuracy = test(densenet, val_loader, cost_function)
test_loss, test_accuracy = test(densenet, test_loader, cost_function)

print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
print('-----------------------------------------------------')


In [0]:
test_loader_copy = test_loader
val_loader_copy = val_loader
train_loader_copy = train_loader
densenet_copy = densenet
samples = 0.
cumulative_loss = 0.
cumulative_accuracy = 0.

densenet_copy.eval() # Strictly needed if network contains layers which has different behaviours between train and tes
predicted_label = list()
real_label = list()
imgPath = list()
with torch.no_grad():
    for batch_idx, (inputs, targets,path) in enumerate(test_loader):
        # Load data into GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = densenet_copy(inputs)

        # Apply the loss
        loss = cost_function(outputs, targets)

        # Better print something
        samples+=inputs.shape[0]
        cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
        _, predicted = outputs.max(1)
        predicted_label.extend(predicted.tolist())
        real_label.extend(targets.tolist())
        imgPath.append(path)
        cumulative_accuracy += predicted.eq(targets).sum().item()


In [0]:
imgPath[0][1][48:55]
C = list()
for i in imgPath:
  for j in i:
    C.append(j[29:49])

UniqueC = set(C)
#print(UniqueC)
ind = list()
for idx in UniqueC:
  index = [i for i, s in enumerate(C) if idx in s]
#  print(index)
  ind.append(index)
  
  
  

In [0]:
from collections import Counter

def get_first_mode(a):
    c = Counter(a)  
    mode_count = max(c.values())
    mode = {key for key, count in c.items() if count == mode_count}
    first_mode = next(x for x in a if x in mode)
    return first_mode

In [0]:
import statistics
from operator import itemgetter 
final_label = list()
final_real_label = list()
#real_im_label = list()
for i in ind:
  #print(itemgetter(*i)(real_label))
  #print(itemgetter(*i)(predicted_label))
  predicted = itemgetter(*i)(predicted_label)
  real_im_label = itemgetter(*i)(real_label)
  #print(get_first_mode(predicted))
  final_label.append(get_first_mode(predicted))
  final_real_label.append(real_im_label[0])




In [0]:
final_real_label == final_label
acc = 0
l = len(final_label)
for i in range(0,l):
  if final_real_label[i] == final_label[i]:
    acc = acc + 1
    
final_acc = acc/len(UniqueC)
print('\t Final Accuracy {:.2f}'.format(final_acc))